In [2]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder \
.master("local[4]") \
.appName("PreprocessOps") \
.config("spark.executor.memory","4g") \
.config("spark.driver.memory","2g") \
.getOrCreate()

In [3]:
df = spark.read \
.option("header","True") \
.option("inferSchema","True") \
.option("sep",",") \
.csv("C:\\Users\\tbura\\data_sets\\simple_data_clear.csv")

In [4]:
df.toPandas().head()

,sirano,isim,yas,meslek,sehir,aylik_gelir
0,1,Cemal,35,Isci,Ankara,3500
1,2,Ceyda,42,Memur,Kayseri,4200
2,3,Timur,30,Müzisyen,Istanbul,9000
3,4,Burcu,29,Pazarlamaci,Ankara,4200
4,5,Yasemin,23,Pazarlamaci,Bursa,4800


## Veri Setine Etiket Ekleme

// Sınıflandırma hedef değişken (etiket-label) yaratmak adına

// Geliri 7000 üstü olanların ekonomik_durumu iyi diyelim.

In [5]:
from pyspark.sql.functions import *

In [6]:
df1 = df.withColumn("ekonomik_durum",
                   when(col("aylik_gelir") >= 7000, "İyi")
                    .when(col("aylik_gelir") < 7000, "Kötü")
                    .otherwise("tanımsız"))

In [7]:
df1.toPandas().head(5)

,sirano,isim,yas,meslek,sehir,aylik_gelir,ekonomik_durum
0,1,Cemal,35,Isci,Ankara,3500,Kötü
1,2,Ceyda,42,Memur,Kayseri,4200,Kötü
2,3,Timur,30,Müzisyen,Istanbul,9000,İyi
3,4,Burcu,29,Pazarlamaci,Ankara,4200,Kötü
4,5,Yasemin,23,Pazarlamaci,Bursa,4800,Kötü


## 1. StringIndexer AŞAMASI

### meslek sütunu için StringIndexer

In [8]:
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler, StandardScaler

In [9]:
meslek_indexer = StringIndexer() \
.setInputCol("meslek") \
.setOutputCol("meslek_index") \
.setHandleInvalid("skip")

meslek_indexer_model = meslek_indexer.fit(df1)
meslek_indexer_df = meslek_indexer_model.transform(df1)

In [10]:
meslek_indexer_df.toPandas().head(15)

,sirano,isim,yas,meslek,sehir,aylik_gelir,ekonomik_durum,meslek_index
0,1,Cemal,35,Isci,Ankara,3500,Kötü,6.0
1,2,Ceyda,42,Memur,Kayseri,4200,Kötü,0.0
2,3,Timur,30,Müzisyen,Istanbul,9000,İyi,2.0
3,4,Burcu,29,Pazarlamaci,Ankara,4200,Kötü,1.0
4,5,Yasemin,23,Pazarlamaci,Bursa,4800,Kötü,1.0
5,6,Ali,33,Memur,Ankara,4250,Kötü,0.0
6,7,Dilek,29,Pazarlamaci,Istanbul,7300,İyi,1.0
7,8,Murat,31,Müzisyen,Istanbul,12000,İyi,2.0
8,9,Ahmet,33,Doktor,Ankara,18000,İyi,3.0
9,10,Muhittin,46,Berber,Istanbul,12000,İyi,7.0


## sehir sütunu için indexer

In [11]:
sehir_indexer = StringIndexer() \
.setInputCol("sehir") \
.setOutputCol("sehir_index")

sehir_indexer_model = sehir_indexer.fit(meslek_indexer_df)
sehir_indexer_df = sehir_indexer_model.transform(meslek_indexer_df)

## 2. OneHotEncoderEstimator AŞAMASI

In [12]:
encoder = OneHotEncoderEstimator() \
.setInputCols(["meslek_index","sehir_index"]) \
.setOutputCols(["meslek_encoded","sehir_encoded"])

In [13]:
encoder_model = encoder.fit(sehir_indexer_df)
encoder_df = encoder_model.transform(sehir_indexer_df)

In [14]:
encoder_df.toPandas().head(15)

,sirano,isim,yas,meslek,sehir,aylik_gelir,ekonomik_durum,meslek_index,sehir_index,meslek_encoded,sehir_encoded
0,1,Cemal,35,Isci,Ankara,3500,Kötü,6.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0)","(1.0, 0.0, 0.0, 0.0, 0.0)"
1,2,Ceyda,42,Memur,Kayseri,4200,Kötü,0.0,5.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 0.0)"
2,3,Timur,30,Müzisyen,Istanbul,9000,İyi,2.0,1.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0)"
3,4,Burcu,29,Pazarlamaci,Ankara,4200,Kötü,1.0,0.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)"
4,5,Yasemin,23,Pazarlamaci,Bursa,4800,Kötü,1.0,2.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 1.0, 0.0, 0.0)"
5,6,Ali,33,Memur,Ankara,4250,Kötü,0.0,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)"
6,7,Dilek,29,Pazarlamaci,Istanbul,7300,İyi,1.0,1.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0)"
7,8,Murat,31,Müzisyen,Istanbul,12000,İyi,2.0,1.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0)"
8,9,Ahmet,33,Doktor,Ankara,18000,İyi,3.0,0.0,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)"
9,10,Muhittin,46,Berber,Istanbul,12000,İyi,7.0,1.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0)"


## 3. VectorAssembler AŞAMASI

In [15]:
assembler = VectorAssembler() \
.setInputCols(["yas","aylik_gelir","meslek_encoded","sehir_encoded"]) \
.setOutputCol("vectorized_features")

In [16]:
# Pandas dataframe head() metodundan satır truncate'i kaldırmak için 
import pandas as pd
pd.set_option('display.max_colwidth', -1)

C:\Users\tbura\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
vector_df = assembler.transform(encoder_df)
vector_df.select("vectorized_features").toPandas().head(15)

,vectorized_features
0,"(35.0, 3500.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0)"
1,"(42.0, 4200.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)"
2,"(30.0, 9000.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)"
3,"(29.0, 4200.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)"
4,"(23.0, 4800.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0)"
5,"(33.0, 4250.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)"
6,"(29.0, 7300.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)"
7,"(31.0, 12000.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)"
8,"(33.0, 18000.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)"
9,"(46.0, 12000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)"


## 4. LabelIndexer AŞAMASI

In [18]:
label_indexer = StringIndexer() \
.setInputCol("ekonomik_durum") \
.setOutputCol("label")

In [19]:
label_indexer_model = label_indexer.fit(vector_df)
label_indexer_df = label_indexer_model.transform(vector_df)

In [20]:
label_indexer_df.select("vectorized_features","label").toPandas().head(15)

,vectorized_features,label
0,"(35.0, 3500.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0)",0.0
1,"(42.0, 4200.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",0.0
2,"(30.0, 9000.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)",1.0
3,"(29.0, 4200.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",0.0
4,"(23.0, 4800.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0)",0.0
5,"(33.0, 4250.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",0.0
6,"(29.0, 7300.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)",1.0
7,"(31.0, 12000.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)",1.0
8,"(33.0, 18000.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",1.0
9,"(46.0, 12000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)",1.0


## 5. StandardScaler AŞAMASI

In [21]:
scaler = StandardScaler() \
.setInputCol("vectorized_features") \
.setOutputCol("features")

In [22]:
scaler_model = scaler.fit(label_indexer_df)
scaler_df = scaler_model.transform(label_indexer_df)

In [23]:
scaler_df.select("features","label").toPandas().head()

,features,label
0,"(5.0082809740601215, 0.7723249167865694, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)",0.0
1,"(6.009937168872146, 0.9267899001438834, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",0.0
2,"(4.292812263480104, 1.9859783574511787, 0.0, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 2.1846572437632577, 0.0, 0.0, 0.0)",1.0
3,"(4.149718521364101, 0.9267899001438834, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)",0.0
4,"(3.29115606866808, 1.0591884573072952, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175, 0.0, 0.0)",0.0


## 6. Train-Test Ayırma AŞAMASI

In [24]:
train_df, test_df = scaler_df.randomSplit([0.8,0.2], seed=142)

In [25]:
train_df.toPandas().head(20)

,sirano,isim,yas,meslek,sehir,aylik_gelir,ekonomik_durum,meslek_index,sehir_index,meslek_encoded,sehir_encoded,vectorized_features,label,features
0,1,Cemal,35,Isci,Ankara,3500,Kötü,6.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(35.0, 3500.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0)",0.0,"(5.0082809740601215, 0.7723249167865694, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)"
1,2,Ceyda,42,Memur,Kayseri,4200,Kötü,0.0,5.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 0.0)","(42.0, 4200.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)",0.0,"(6.009937168872146, 0.9267899001438834, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)"
2,4,Burcu,29,Pazarlamaci,Ankara,4200,Kötü,1.0,0.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(29.0, 4200.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",0.0,"(4.149718521364101, 0.9267899001438834, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)"
3,5,Yasemin,23,Pazarlamaci,Bursa,4800,Kötü,1.0,2.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 1.0, 0.0, 0.0)","(23.0, 4800.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0)",0.0,"(3.29115606866808, 1.0591884573072952, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175, 0.0, 0.0)"
4,7,Dilek,29,Pazarlamaci,Istanbul,7300,İyi,1.0,1.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0)","(29.0, 7300.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)",1.0,"(4.149718521364101, 1.610849112154845, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.1846572437632577, 0.0, 0.0, 0.0)"
5,8,Murat,31,Müzisyen,Istanbul,12000,İyi,2.0,1.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0)","(31.0, 12000.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)",1.0,"(4.435906005596108, 2.6479711432682382, 0.0, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 2.1846572437632577, 0.0, 0.0, 0.0)"
6,11,Hicaziye,47,Tuhafiyeci,Ankara,4800,Kötü,5.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(47.0, 4800.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",0.0,"(6.725405879452164, 1.0591884573072952, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)"
7,12,Harun,43,Tornacı,Ankara,4200,Kötü,4.0,0.0,"(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(43.0, 4200.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",0.0,"(6.15303091098815, 0.9267899001438834, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175, 0.0, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)"
8,13,Hakkı,33,Memur,Çorum,3750,Kötü,0.0,3.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 1.0, 0.0)","(33.0, 3750.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0)",0.0,"(4.722093489828115, 0.8274909822713244, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175, 0.0)"
9,14,Gülizar,37,Doktor,İzmir,14250,İyi,3.0,4.0,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 1.0)","(37.0, 14250.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0)",1.0,"(5.294468458292129, 3.1444657326310326, 0.0, 0.0, 0.0, 2.8419928002940256, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8729833462074175)"


In [26]:
test_df.toPandas().head(20)

,sirano,isim,yas,meslek,sehir,aylik_gelir,ekonomik_durum,meslek_index,sehir_index,meslek_encoded,sehir_encoded,vectorized_features,label,features
0,3,Timur,30,Müzisyen,Istanbul,9000,İyi,2.0,1.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0)","(30.0, 9000.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)",1.0,"(4.292812263480104, 1.9859783574511787, 0.0, 0.0, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 2.1846572437632577, 0.0, 0.0, 0.0)"
1,6,Ali,33,Memur,Ankara,4250,Kötü,0.0,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(33.0, 4250.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",0.0,"(4.722093489828115, 0.9378231132408343, 2.41522945769824, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)"
2,9,Ahmet,33,Doktor,Ankara,18000,İyi,3.0,0.0,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(33.0, 18000.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",1.0,"(4.722093489828115, 3.9719567149023574, 0.0, 0.0, 0.0, 2.8419928002940256, 0.0, 0.0, 0.0, 1.9364916731037087, 0.0, 0.0, 0.0, 0.0)"
3,10,Muhittin,46,Berber,Istanbul,12000,İyi,7.0,1.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0)","(46.0, 12000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)",1.0,"(6.58231213733616, 2.6479711432682382, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.1846572437632577, 0.0, 0.0, 0.0)"


## 7. Basit Bir Makine Öğrenmesi Modeli

In [27]:
from pyspark.ml.classification import LogisticRegression

In [28]:
lr_object = LogisticRegression() \
.setFeaturesCol("features") \
.setLabelCol("label") \
.setPredictionCol("prediction")

In [29]:
lr_model = lr_object.fit(train_df)

In [30]:
result_df = lr_model.transform(test_df)

In [32]:
result_df.select("label","prediction").toPandas().head()

,label,prediction
0,1.0,1.0
1,0.0,0.0
2,1.0,1.0
3,1.0,1.0
